# Projet : Système de recommandation de musiques

In [ ]:
import os
from google.colab import drive
import tarfile
import h5py
import pandas as pd

drive.mount('/content/drive')

# Change the current working directory to the path of Google Cloud Drive
path="/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset"
os.listdir(path)

Mounted at /content/drive


['A',
 'B',
 'user_interactions.gsheet',
 'recommendations.csv',
 'user_interactions.csv',
 'user_item_matrix.csv',
 'songs_data.csv',
 'songs_data_cleaned.csv',
 'user_item_weighted_matrix.csv',
 'filtered_recommendations.csv',
 'train_triplets.txt',
 'all_users_filtered.gsheet',
 'all_users.csv',
 'all_users_filtered.csv',
 'user_interactions_with_duration_hotness.csv',
 'user_item_matrix.npy',
 'item_cooccurrence_matrix.npy',
 'ease_weights.npy',
 'filtered_recommendations_real_users.csv',
 'user_item_interactions.csv']

## Chargement des fichiers dataset utilisateurs

In [ ]:
import pandas as pd

# Spécifiez le chemin de votre fichier
file_path = '/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/train_triplets.txt'

# Lire le fichier dans un DataFrame
df = pd.read_csv(file_path, sep='\t', header=None, names=['user_id', 'song_id', 'play_count'])

# Spécifiez le chemin de sortie pour le fichier CSV
csv_output_path = '/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/all_users.csv'

# Sauvegarder le DataFrame dans un fichier CSV
df.to_csv(csv_output_path, index=False)
print('Finish')

Finish


## Filtrage des utilisateurs

In [ ]:
import pandas as pd

# Charger les données des chansons
songs_data_path = '/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/songs_data_cleaned.csv'
songs_df = pd.read_csv(songs_data_path)

# Créer un ensemble avec les identifiants des chansons de votre sous-ensemble
song_subset = set(songs_df['song_id'])

# Charger les données des utilisateurs
user_data_path = '/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/all_users.csv'
user_df = pd.read_csv(user_data_path)

# Filtrer les lignes où song_id est dans song_subset
filtered_df = user_df[user_df['song_id'].isin(song_subset)]

# Compter le nombre de chansons uniques écoutées par chaque utilisateur
user_song_counts = filtered_df.groupby('user_id')['song_id'].nunique()

# Filtrer pour ne garder que les utilisateurs qui ont écouté plus de 5 chansons différentes
filtered_users = user_song_counts[user_song_counts > 5].index

# Créer un nouveau DataFrame avec seulement ces utilisateurs
final_filtered_df = filtered_df[filtered_df['user_id'].isin(filtered_users)]

# Sauvegarder le DataFrame final
final_filtered_df.to_csv('/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/all_users_filtered.csv', index=False)


## Affichage nombre d'utilisateurs

In [ ]:
import pandas as pd

# Charger le fichier all_users_filtered.csv
filtered_data_path = '/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/all_users_filtered.csv'
filtered_df = pd.read_csv(filtered_data_path)

# Compter le nombre d'utilisateurs uniques
unique_user_count = filtered_df['user_id'].nunique()

print(f"Nombre d'utilisateurs uniques : {unique_user_count}")


Nombre d'utilisateurs uniques : 9897


## Prétraitement

In [ ]:
import os
import h5py
import pandas as pd
from multiprocessing import Pool, current_process

root_path = '/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset'
aggregated_data = []

def read_h5_file(file_path):
    with h5py.File(file_path, 'r') as file:
        # Exemple d'extraction des données
        song_id = file['metadata/songs']['song_id'][0].decode('UTF-8')
        artist_name = file['metadata/songs']['artist_name'][0].decode('UTF-8')
        song_hotttnesss = file['metadata/songs']['song_hotttnesss'][0]
        duration = file['analysis/songs']['duration'][0]

        # Ajouter les données extraites à la liste aggregated_data
        aggregated_data.append({
            'song_id': song_id,
            'artist_name': artist_name,
            'song_hotttnesss': song_hotttnesss,
            'duration': duration
        })
        return {
            'song_id': song_id,
            'artist_name': artist_name,
            'song_hotttnesss': song_hotttnesss,
            'duration': duration
        }

def process_files(file_paths):
    data = []
    process_id = current_process().name  # Identifiant du processus
    for index, file_path in enumerate(file_paths):
        data.append(read_h5_file(file_path))
        if index % 100 == 0:  # Affiche un message tous les 100 fichiers
            print(f"{process_id}: Processed {index} files")
    return data

# Créer une liste de tous les chemins de fichiers .h5
all_files = [os.path.join(root, file) for root, dirs, files in os.walk(root_path) for file in files if file.endswith('.h5')]

# Nombre de processus à utiliser
num_processes = 400 # Vous pouvez ajuster cela en fonction de votre CPU

# Diviser la liste des fichiers en sous-ensembles pour chaque processus
chunk_size = len(all_files) // num_processes
file_chunks = [all_files[i:i + chunk_size] for i in range(0, len(all_files), chunk_size)]

# Utiliser multiprocessing pour traiter les fichiers en parallèle
with Pool(num_processes) as pool:
    results = pool.map(process_files, file_chunks)

# Aplatir la liste des résultats
aggregated_data = [item for sublist in results for item in sublist]

# Convertir en DataFrame et sauvegarder
df_songs = pd.DataFrame(aggregated_data)
df_songs.to_csv(os.path.join(root_path, "songs_data.csv"), index=False)


## Nettoyage et conversion des données

In [ ]:
import pandas as pd

# Charger le DataFrame
df_songs = pd.read_csv('/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/songs_data.csv')

# Fonction pour convertir la durée en secondes en format minutes:secondes
def convert_duration(seconds):
    minutes = int(seconds // 60)
    seconds = int(seconds % 60)
    return f"{minutes}:{seconds:02d}"

# Appliquer la fonction de conversion à la colonne 'duration'
df_songs['duration'] = df_songs['duration'].apply(convert_duration)

# Nettoyage des données
# Supprimer les lignes où song_hotttnesss ou duration sont NaN
df_songs.dropna(subset=['song_hotttnesss', 'duration'], inplace=True)

# Sauvegarder le DataFrame nettoyé pour une utilisation ultérieure
df_songs.to_csv('/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/songs_data_cleaned.csv', index=False)


## Création des données interactions utilisateurs-musiques

In [ ]:
import pandas as pd

# Charger les données des chansons nettoyées
df_songs = pd.read_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/songs_data_cleaned.csv')

# Charger les données d'écoute des utilisateurs filtrées
df_users_filtered = pd.read_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/all_users_filtered.csv')

# Regrouper les données par utilisateur et créer une liste des chansons écoutées par chaque utilisateur
grouped_user_songs = df_users_filtered.groupby('user_id')['song_id'].apply(list)

# Initialiser les listes pour stocker les données des utilisateurs
user_interactions = []

# Pour chaque utilisateur, créer des interactions avec les chansons qu'il a écoutées
for user_id, song_ids in grouped_user_songs.items():
    # Obtenir les informations sur les chansons écoutées
    user_songs_info = df_songs[df_songs['song_id'].isin(song_ids)]

    # Ajouter les informations à la liste
    for _, song_row in user_songs_info.iterrows():
        user_interactions.append({
            'user_id': user_id,
            'song_id': song_row['song_id'],
            'artist_name': song_row['artist_name'],
            'duration': song_row['duration'],
            'song_hotttnesss': song_row['song_hotttnesss']
        })

# Créer un DataFrame pour les interactions utilisateurs
df_user_interactions = pd.DataFrame(user_interactions)

# Sauvegarder le DataFrame des interactions
df_user_interactions.to_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/user_interactions_with_duration_hotness.csv', index=False)


## Implémentation de l'Autoencodeur Superficiel EASE sur MSD
## Transformation des données d'interactions en une matrice utilisateur-item

In [ ]:
import pandas as pd
import numpy as np

# Charger les données d'interactions enrichies
df_interactions = pd.read_csv('/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/user_interactions_with_duration_hotness.csv')

# Fonction pour convertir la durée en secondes
def convert_duration_to_seconds(duration_str):
    minutes, seconds = map(int, duration_str.split(':'))
    return minutes * 60 + seconds

# Convertir la durée en secondes et calculer le poids
df_interactions['duration_seconds'] = df_interactions['duration'].apply(convert_duration_to_seconds)
df_interactions['weight'] = df_interactions['duration_seconds'] * df_interactions['song_hotttnesss']

# Mappage des identifiants aux indices de la matrice
user_ids = df_interactions['user_id'].unique()
song_ids = df_interactions['song_id'].unique()
user_id_to_index = {user_id: index for index, user_id in enumerate(user_ids)}
song_id_to_index = {song_id: index for index, song_id in enumerate(song_ids)}

# Initialiser la matrice globale
num_users = len(user_ids)
num_songs = len(song_ids)
global_matrix = np.zeros((num_users, num_songs))

# Traitement par batches
batch_size = 10000  # Ajustez en fonction de votre mémoire disponible
for start in range(0, len(df_interactions), batch_size):
    end = min(start + batch_size, len(df_interactions))
    batch = df_interactions.iloc[start:end]
    for _, row in batch.iterrows():
        user_idx = user_id_to_index[row['user_id']]
        song_idx = song_id_to_index[row['song_id']]
        weight = row['weight']
        global_matrix[user_idx, song_idx] += weight

# Convertir la matrice globale en DataFrame et réinitialiser l'index
global_df = pd.DataFrame(global_matrix, index=user_ids, columns=song_ids).reset_index()
global_df.rename(columns={'index': 'user_id'}, inplace=True)

# Réordonner les colonnes pour que 'user_id' soit la première
column_order = ['user_id'] + list(song_ids)
global_df = global_df[column_order]

# Enregistrer le DataFrame dans un fichier CSV
global_df.to_csv('/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/user_item_interactions.csv', index=False)

print("Le fichier CSV user_item_interactions.csv a été créé avec succès.")


## Construction de la Matrice de Cooccurrence des Items

In [ ]:
import pandas as pd
import numpy as np

# Charger la matrice utilisateur-item pondérée depuis un fichier .csv
user_item_matrix = pd.read_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/user_item_interactions.csv')

# Calculer la matrice de cooccurrence des items
item_cooccurrence_matrix = np.dot(user_item_matrix.T, user_item_matrix)
np.fill_diagonal(item_cooccurrence_matrix, 0)  # Remplir la diagonale avec des zéros
pd.save_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/item_cooccurrence_matrix.csv', item_cooccurrence_matrix)

## Résolution de la Régression Ridge

In [ ]:
import pandas as pd
import numpy as np

def ease_regression(item_cooccurrence_matrix, lambda_):
    """
    Résout la régression ridge pour le modèle EASE.

    :param item_cooccurrence_matrix: Matrice de cooccurrence des items.
    :param lambda_: Paramètre de régularisation.
    :return: Matrice de poids du modèle EASE.
    """
    m, n = item_cooccurrence_matrix.shape
    identity_matrix = np.identity(n)
    ridge_reg_matrix = item_cooccurrence_matrix + lambda_ * identity_matrix
    inverse_matrix = np.linalg.inv(ridge_reg_matrix)
    b_matrix = inverse_matrix / -np.diag(inverse_matrix)
    np.fill_diagonal(b_matrix, 0)
    return b_matrix

# Charger la matrice de cooccurrence des items depuis un fichier .csv
item_cooccurrence_matrix = pd.read_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/item_cooccurrence_matrix.csv', index_col=0)

# Conversion de DataFrame en NumPy Array pour la régression
item_cooccurrence_matrix_np = item_cooccurrence_matrix.to_numpy()

# Appliquer la régression ridge pour obtenir la matrice de poids du modèle EASE
lambda_ = 100  # Paramètre de régularisation
ease_weights_np = ease_regression(item_cooccurrence_matrix_np, lambda_)

# Convertir la matrice de poids NumPy en DataFrame
ease_weights_df = pd.DataFrame(ease_weights_np, index=item_cooccurrence_matrix.columns, columns=item_cooccurrence_matrix.columns)

# Enregistrer la matrice de poids EASE dans un fichier CSV
ease_weights_df.to_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/ease_weights.csv')


## Génération des Recommandations

In [ ]:
import pandas as pd
import numpy as np

# Charger les matrices
user_item_matrix = np.load('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/user_item_matrix.npy')
item_cooccurrence_matrix = np.load('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/item_cooccurrence_matrix.npy')
ease_weights = np.load('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/ease_weights.npy')

# Générer des recommandations pour chaque utilisateur
user_recommendations = np.dot(user_item_matrix, ease_weights)

# Supposons que vous ayez des listes user_ids et song_ids
# df_recommendations = pd.DataFrame(user_recommendations, index=user_ids, columns=song_ids)

# Exemple de génération d'index et de colonnes (à remplacer par vos données réelles)
user_ids = [f'user_{i}' for i in range(user_item_matrix.shape[0])]
song_ids = [f'song_{i}' for i in range(user_item_matrix.shape[1])]

# Convertir les recommandations en DataFrame
df_recommendations = pd.DataFrame(user_recommendations, index=user_ids, columns=song_ids)

# Filtrer et limiter les recommandations pour chaque utilisateur
filtered_recommendations = pd.DataFrame()

for user_id in df_recommendations.index:
    # Récupérer les chansons déjà écoutées
    listened_songs = set(df_interactions[df_interactions['user_id'] == user_id]['song_id'])

    # Récupérer les recommandations pour cet utilisateur
    user_recs = df_recommendations.loc[user_id]

    # Filtrer les chansons déjà écoutées
    user_recs_filtered = user_recs[~user_recs.index.isin(listened_songs)]

    # Trier et limiter à 100 recommandations
    top_100_recs = user_recs_filtered.sort_values(ascending=False).head(100)

    # Ajouter au DataFrame
    filtered_recommendations[user_id] = top_100_recs

# Transposer le DataFrame pour un meilleur format
filtered_recommendations = filtered_recommendations.T

# Sauvegarder les recommandations filtrées
filtered_recommendations.to_csv('/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/filtered_recommendations_real_users.csv')


## Évaluation et Ajustement

In [ ]:
import pandas as pd
import numpy as np

# Chemin des fichiers
user_item_matrix_path = '/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/user_item_interactions.csv'
songs_data_path = '/content/drive/MyDrive/Laval/Projet_SIGNAL/MillionSongSubset/songs_data_cleaned.csv'

# Charger la matrice utilisateur-item
user_item_matrix_np = np.load(user_item_matrix_path)

# Supposons que vous ayez les listes user_ids et song_ids
# user_ids = [...]
# song_ids = [...]

# Convertir la matrice NumPy en DataFrame Pandas
user_item_matrix = pd.DataFrame(user_item_matrix_np, index=user_ids, columns=song_ids)

# Charger les données des chansons
df_songs = pd.read_csv(songs_data_path)

# Fonction pour convertir la durée en secondes
def convert_duration_to_seconds(duration):
    if isinstance(duration, float) or isinstance(duration, int):
        return duration
    else:
        minutes, seconds = map(int, duration.split(':'))
        return minutes * 60 + seconds

# Convertir la durée et remplir les valeurs manquantes pour la hotness
df_songs['duration_in_seconds'] = df_songs['duration'].apply(convert_duration_to_seconds)
df_songs['song_hotttnesss'] = df_songs['song_hotttnesss'].fillna(0)

# Normaliser les données de durée et de hotness
max_duration = df_songs['duration_in_seconds'].max()
min_duration = df_songs['duration_in_seconds'].min()
df_songs['normalized_duration'] = (df_songs['duration_in_seconds'] - min_duration) / (max_duration - min_duration)

max_hotness = df_songs['song_hotttnesss'].max()
min_hotness = df_songs['song_hotttnesss'].min()
df_songs['normalized_hotness'] = (df_songs['song_hotttnesss'] - min_hotness) / (max_hotness - min_hotness)

# Combiner durée normalisée et hotness normalisée pour créer une mesure de pertinence
df_songs['combined_relevance'] = (df_songs['normalized_duration'] + df_songs['normalized_hotness']) / 2

# Créer un dictionnaire de pertinence
relevance_dict = df_songs.set_index('song_id')['combined_relevance'].to_dict()

# Transformer la matrice utilisateur-item en dictionnaire d'interactions avec pertinence normalisée
actual_interactions = {
    user_id: {song_id: relevance_dict.get(song_id, 0)
              for song_id in user_item_matrix.columns[(user_item_matrix.loc[user_id] > 0)]}
    for user_id in user_item_matrix.index
}

# Affichage des données de pertinence normalisées
print(relevance_dict)


## Calcul DCG

In [ ]:
def calculate_dcg(recommended_items, relevance, k):
    dcg = 0
    for i in range(k):
        item_relevance = relevance.get(recommended_items[i], 0)
        dcg += item_relevance / np.log2(i + 2)  # i+2 car l'index commence à 0 et le log de 1 est 0
    return dcg

def calculate_ndcg(recommended_items, relevance, k):
    dcg = calculate_dcg(recommended_items, relevance_dict, k)
    idcg = calculate_dcg(sorted(recommended_items, key=lambda x: relevance_dict.get(x, 0), reverse=True), relevance_dict, k)
    return dcg / idcg if idcg > 0 else 0

## Affichage score NDCG

In [ ]:
import pandas as pd
# Charger les recommandations EASE
df_recommendations = pd.read_csv('/content/drive/MyDrive/Projet_SIGNAL/MillionSongSubset/filtered_recommendations_real_users.csv', index_col=0)

ndcg_scores = []
k = 100  # Nombre de recommandations à considérer

for user_id in df_recommendations.index:
    recommended_items = df_recommendations.loc[user_id].sort_values(ascending=False).index.tolist()[:k]
    ndcg_score = calculate_ndcg(recommended_items, relevance_dict, k)
    ndcg_scores.append(ndcg_score)

# Calculate NDCG score
ease_average_ndcg = np.mean(ndcg_scores)
ease_variance_ndcg = np.var(ndcg_scores)
print(f"Mean : {ease_average_ndcg:.5f} - Variance : {ease_variance_ndcg:.5f}")